# Calculations - Flerovium kinetics

In [ ]:
%run SRIM_Ca48.ipynb
%run GenerateStoppingPower.ipynb

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.linalg
from matplotlib import colors
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_pdf import PdfPages
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

color_list = plt.rcParams['axes.prop_cycle'].by_key()['color'];
#Customising style
#mpl.rc('lines', linewidth=2, color='r')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['font.size'] = 24
#mpl.rcParams['savefig.format'] = "eps"
#mpl.rcParams['savefig.dpi'] = 100
#mpl.rcParams['text.usetex'] = True
#mpl.rcParams['figure.figsize'] = (16,9)

## Initialise general parameters

In [ ]:
e_tank1 = 4.774 #MeV/u
e_tank2 = 5.893 #MeV/u
#e_tank2 = 6.02 #MeV/u
A_beam = 48

## Pu-target for flerovium production

Ca-48 on Pu-target.

Beam energies at middle of target should be aimed for: 

* 240 MeV = ~5 MeV/u

See Dirk's folder.

### Basic settings

In [ ]:
e_tank = e_tank2
target = "PuO2"
degrader = "Ti"
backing = "Ti"

In [ ]:
print("Beam energy:", e_tank*A_beam, "MeV")

### Creating stopping power functions

"Targets":

In [ ]:
layers = []

# Ti
density = 4.506e3 #mg/cm3 (from wikipedia)
densities.append(density)
# Construct a layer of Ti, arbitrary width (as it is not relevant for these calcs)
layers.append(Layer({
    'Ti': {
        'stoich': 1
        #'E_d': 35.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    }
}, density=1e-3*density, width=10000.0))

# PuO2
density = 11.5 #g/cm3 (from wikipedia)
densities.append(density)
# Construct a layer of U, arbitrary width (as it is not relevant for these calcs)
layers.append(Layer({
    'U': {
        'stoich': 1
        #'E_d': 35.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    }
}, density=density, width=10000.0))

# Calculating mean/effective nuclear charge for reference layer and desired layer.
Z_UO2 = (92+2*8)/3.
Z_PuO2 = (94+2*8)/3.

layer_names = ["Ti", "PuO2"]

"Ion":

In [ ]:
# Construct ion
A, E_per_A_max = 48, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('Ca', energy=energy_max, mass=A)

Stopping power functions:

In [ ]:
Ca48_GSP = {}

Ca48_GSP[layer_names[0]] = GenerateStoppingPower(layers[0], ion)
Ca48_GSP[layer_names[1]] = GenerateStoppingPower(layers[1], ion, Z=Z_PuO2, Z_ref=Z_UO2)

### Thickness - degrader, backing, target

In [ ]:
#degrader_thickness = 0.76 # mg/cm2
#degrader_thickness = 1e4*(degrader_thickness / 2.70e3) #µm
degrader_thickness = 5.6 #µm & 5.05 µm

#backing_thickness = 40 # µg/cm2 (according to Dirk)
#backing_thickness = 1e4*(backing_thickness/2.15e6) # µm
backing_thickness = 2.24 #µm

target_thickness = 0.91 #mg/cm2 (according to 'Target Considerations Summary' file)
target_thickness = 1e4*(target_thickness/11.5e3) # µm
print("Target thickness: ", target_thickness)

### Traversing material and calculating energy

In [ ]:
E_of_X = [] #energies
X = [] # distance
E = e_tank*A_beam
E_of_X.append(E)
X.append(0)
print("E("+str(X[-1])+" µm)=", E_of_X[-1], 'MeV', E_of_X[-1]/A, 'MeV/u')

# degrader
E = E - Ca48_GSP[degrader].GetElossAfterX(E, degrader_thickness)
#e_loss_x(E, degrader_thickness, degrader)
E_of_X.append(E)
X.append(X[-1]+degrader_thickness)
print("E("+str(X[-1])+" µm)=", E_of_X[-1], 'MeV', E_of_X[-1]/A, 'MeV/u')

# backing
E = E - Ca48_GSP[backing].GetElossAfterX(E, backing_thickness)
#e_loss_x(E, backing_thickness, backing)
E_of_X.append(E)
X.append(X[-1]+backing_thickness)
print("E("+str(X[-1])+" µm)=", E_of_X[-1], 'MeV', E_of_X[-1]/A, 'MeV/u')

# target, middle and end
E = E - Ca48_GSP[target].GetElossAfterX(E, target_thickness*0.5)
#e_loss_x(E, target_thickness*0.5, target)
E_of_X.append(E)
X.append(X[-1]+target_thickness*0.5)
print("E("+str(X[-1])+" µm)=", E_of_X[-1], 'MeV', E_of_X[-1]/A, 'MeV/u')
E = E - Ca48_GSP[target].GetElossAfterX(E, target_thickness*0.5)
E_of_X.append(E)
X.append(X[-1]+target_thickness*0.5)
print("E("+str(X[-1])+" µm)=", E_of_X[-1], 'MeV', E_of_X[-1]/A, 'MeV/u')

E_of_X = np.asarray(E_of_X)
X = np.asarray(X)

In [ ]:
stop_labels = ['start_degrader', 'start_backing', 'start_target', 'middle_target', 'end_target']

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(20,10))
axes[0].plot(X, E_of_X, label='E(X)')
axes[0].set_xlabel("Traversed material (µm)")
axes[0].set_ylabel('Energy (MeV))')
axes[1].plot(stop_labels, E_of_X, label='E(X)')
plt.xticks(rotation=90)
axes[1].set_xlabel("Traversed material")
axes[1].set_ylabel('Energy (MeV)')
ax2 = axes[1].twinx()
ax2.plot(stop_labels, E_of_X/A, label='E(X)/u')
ax2.set_ylabel('Energy (MeV/u)')
plt.show()

# Fusion: flerovium excitation and kinetic energies

In [ ]:
from pyne import data as pyne_data

In [ ]:
A_compound = 253
masses = {"Ca48": pyne_data.atomic_mass("Ca48"), "No255": pyne_data.atomic_mass("No255"), "Pb207": pyne_data.atomic_mass("Pb207")}
for m in masses:
    masses[m] *= 931.494 #MeV/c^2
masses

Looping target

In [ ]:
ind_start = 2
E_compound, E_exc = [], []

for i in range(ind_start, len(E_of_X)):
    E_beam = E_of_X[i]
    E_kin_compound =  E_beam * (A_beam/A_compound)
    excitation_energy = masses["Ca48"] + masses["Pb207"] - masses["No255"] + E_beam - E_kin_compound    
    E_compound.append(E_kin_compound)
    E_exc.append(excitation_energy)

print("Fusion at: ", stop_labels[ind_start:])
print("Compound kinetic energy: ", E_compound)
print("Compound excitation energy: ", E_exc)

## Energy loss in gas filled separator

In [ ]:
tasca_length = 4 * 1e6 # 4 m in µm (ballpark figure taken from CAD drawing)

In [ ]:
No253 = {}

In [ ]:
# Construct a layer of He, arbitrary width (as it is not relevant for these calcs)
pressure = 0.8 #mbar
density = 0.1786 #g/L at STP
density *= 1e-3 #g/cm^3
density *= pressure*1e-3
print("density:", density, "g/cm^3")
print("NOTE, too low density for SRIM to handle it seems")
layer = Layer({
    'He': {
        'stoich': 1,
        #'E_d': 35.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    }
}, density=density, width=10000.0)

Average charge state of No in TASCA (see Ulrika's thesis, p. 33-34)

In [ ]:
v0 = 2.19e6 #m/s, Bohr velocity
c = 3e8 #light speed, m/s
m_No = 931.494*253
v = np.sqrt(np.divide(np.asarray(E_compound), m_No))*c
print("Velocity:", v)
Z = 102
x = np.divide(v, v0)*Z**(1./3)
q_avg = 0.641*x-0.235+0.517*np.sin(2*np.pi/32*(0.641*x-0.235)-74.647)
print("Average charge state:", q_avg)
q_avg = q_avg[1]

In [ ]:
# Construct ion
A, E_per_A_max = 253, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('U', energy=energy_max, mass=A)

# FAILS WITH DENSITY HERE
#No253["He"] = GenerateStoppingPower(layer, ion, z_ion=102)

[SRIM for No in He](SRIM_No_in_He.png)
(SRIM_No_in_He.png)

In [ ]:
dEdX = 5.995e-4+4.607e-5 #MeV/mm
dEdX *= 1e-3

In [ ]:
E_compound_TASCA = []

for i, e in enumerate(E_compound):
    #e_stop = (No253["He"].f_dEdX(e))*(q_avg/Z) #normalise with average charge
    e_stop = (dEdX)*(q_avg/Z) #normalise with average charge
    E_compound_TASCA.append(e-e_stop*tasca_length)
    
print("Fusion at: ", stop_labels[ind_start:])
print("Energy after TASCA:", E_compound_TASCA)

How do you really estimate this?

In [ ]:
#E_compound_TASCA = E_compound

### Implantation energy

In [ ]:
# Construct a layer of SiO2, arbitrary width (as it is not relevant for these calcs)
layer = Layer({
    'Si': {
        'stoich': 1,
        #'E_d': 35.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    },
    'O': {
        'stoich': 2,
        #'E_d': 20.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    }
}, density=2.32, width=10000.0)

In [ ]:
# Construct ion
A, E_per_A_max = 253, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('U', energy=energy_max, mass=A)

No253["Si"] = GenerateStoppingPower(layer, ion, z_ion=102)

Loss in deadlayer (2 µm) and detected implantation energies:

In [ ]:
E_det = []
for i, e in enumerate(E_compound_TASCA):
    E_loss = No253["Si"].GetElossAfterX(e, 2)
    E_det.append(e-E_loss)

print("Fusion at: ", stop_labels[ind_start:])
print("Implant energy detected:", E_det)